In [1]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json, math, time
from tqdm.notebook import tqdm_notebook
import tmdbsimple as tmdb

In [2]:
# open json file
with open('/Users/ramos/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
login.keys()

dict_keys(['api-key'])

In [3]:
tmdb.API_key = login['api-key']

In [4]:
# API calls from Part 1 of this project
import os
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['title_akas.csv.gz', 'title_basics.csv.gz', 'title_ratings.csv.gz']

In [5]:
# Define years to get
YEARS_TO_GET = [2000,2001]

In [6]:
#function to get movie rating
def get_movie_with_rating(movie_id): 
    
    #get the movie object for the current id
    movie = tmdb.Movies(movie_id)
    
    #save the .info .releases dictionaries
    movie_info = movie.info()
    releases = movie.releases()
    
    #loop through countries in releases
    for c in releases['countries']:
        #if the country abbreviation==US
        if c['iso_3166_1'] =='US':
            #save a "certification" key in the info dict with the certification
            movie_info['certification'] = c['certification']
            
    return movie_info

In [7]:
#function to create a new json file 
def write_json(new_data, filename): 
        """Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
        
        with open(filename,'r+') as file:
            #first we load existing data into a dict.
            file_data = json.load(file)
            
            #choose extend or append
            if (type(new_data) == list) & (type(file_data) == list):
                file_data.extend(new_data)
            else:
                file_data.append(new_data)
                
        #sets file's current position at offset.
            file.seek(0)
            
        #convert back to json.
            json.dump(file_data, file)

In [9]:
# OUTER loop
for YEAR in tqdm_notebook(YEARS_TO_GET,desc='YEARS',position=0):
    
    #defining the JSON file to store results for year
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    
    #check if file exists
    file_exists = os.path.isfile(JSON_FILE)
    
    #if it does not exist: create it
    if file_exists == False:
        
        #save an empty dict with just "imdb_id" to the new json file.
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)
            
    #load in the dataframe from project part 1 as basics:
    basics = pd.read_csv('Data/title_basics.csv.gz')
    
    #saving new year as the current df
    df = basics.loc[ basics['startYear']==YEAR].copy()
    
    #saving movie ids to list
    movie_ids = df['tconst'].copy()
   
    #load existing data from json into a dataframe called "previous_df"
    previous_df = pd.read_json(JSON_FILE)
    
    # filter out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]
    
    #INNER Loop
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
        #attempt to retrieve then data for the movie id
        try:
            temp = get_movie_with_rating(movie_id) 
        
            #append/extend results to existing file using a pre-made function
            write_json(temp,JSON_FILE)
            
            #short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)
            
        #if it fails,  make a dict with just the id and None for certification.
        except Exception as e:
            continue
      
    #open json file and save as csv
    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)
    

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/1403 [00:00<?, ?it/s]

Movies from 2001:   0%|          | 0/1517 [00:00<?, ?it/s]

In [ ]:
# Load dataframe for year 2000
df_2000 = pd.read_csv('Data/final_tmdb_data_2000.csv.gz')
df_2000.head()